In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [2]:
def browser():
    #Initializing the webdriver
    options = webdriver.ChromeOptions()

    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')

    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="/Users/joe/Sprint2_MMA460/scraping-glassdoor-selenium-master/chromedriver", options=options)
    driver.set_window_size(1120, 1000)
    print('Initialized browser')

In [3]:
def sign_in():

    url = 'https://www.glassdoor.com/profile/login_input.htm'
    browser.get(url)
    
    time.sleep(2)

    # import pdb;pdb.set_trace()

    email_field = browser.find_element_by_name('username')
    password_field = browser.find_element_by_name('password')
    submit_btn = browser.find_element_by_xpath('//button[@type="submit"]')

    email_field.send_keys('pejok98777@hxqmail.com')
    password_field.send_keys('gD1234567Gd')
    submit_btn.click()

    time.sleep(1)
    
    print('Signed In')

In [10]:
def get_jobs_GD(keyword, num_jobs, seniority, verbose):
    
    #seniority Types:
        # internship
        # entrylevel
        # midseniorlevel
        # director
        # executive
        # all
    
    seniority = seniority
    
    # BROWSER
    #Initializing the webdriver
    options = webdriver.ChromeOptions()

    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')

    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="/Users/joe/Sprint2_MMA460/scraping-glassdoor-selenium-master/chromedriver", options=options)
    driver.set_window_size(1120, 1000)
    print('Initialized browser')
    
    # -=-=-=-=-=-=-
    
    #SIGNING IN 
    url = 'https://www.glassdoor.com/profile/login_input.htm'
    driver.get(url)
    
    time.sleep(2)

    # import pdb;pdb.set_trace()

    email_field = driver.find_element_by_name('username')
    password_field = driver.find_element_by_name('password')
    submit_btn = driver.find_element_by_xpath('//button[@type="submit"]')

    email_field.send_keys('pejok98777@hxqmail.com')
    password_field.send_keys('gD1234567Gd')
    submit_btn.click()

    time.sleep(1)
    
    print('Signed In')
    
    # -=-=-=-=-=-=-
    
    # DATA MINING SECTION
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''

    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword=' + keyword + '&locT=S&locId=1282&locKeyword=North%20Carolina&jobType=all&fromAge=30&minSalary=0&includeNoSalaryJobs=true&radius=25&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=' + seniority + '&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    
    driver.get(url)
    jobs = []
    
    print('Opened Page')
        
        
    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click() 
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_class_name("SVGInline modal_closeIcon").click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray small salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
            

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [91]:
#This line will open a new chrome window and start the scraping.
df = get_jobs_GD("analyst", 1200, False)
df

Initialized browser
Signed In
Opened Page
Progress: 0/1200
Progress: 1/1200
Progress: 2/1200
Progress: 3/1200
Progress: 4/1200
Progress: 5/1200
Progress: 6/1200
Progress: 7/1200
Progress: 8/1200
Progress: 9/1200
Progress: 10/1200
Progress: 11/1200
Progress: 12/1200
Progress: 13/1200
Progress: 14/1200
Progress: 15/1200
Progress: 16/1200
Progress: 17/1200
Progress: 18/1200
Progress: 19/1200
Progress: 20/1200
Progress: 21/1200
Progress: 22/1200
Progress: 23/1200
Progress: 24/1200
Progress: 25/1200
Progress: 26/1200
Progress: 27/1200
Progress: 28/1200
Progress: 29/1200
Progress: 30/1200
Progress: 31/1200
Progress: 32/1200
Progress: 33/1200
Progress: 34/1200
Progress: 35/1200
Progress: 36/1200
Progress: 37/1200
Progress: 38/1200
Progress: 39/1200
Progress: 40/1200
Progress: 41/1200
Progress: 42/1200
Progress: 43/1200
Progress: 44/1200
Progress: 45/1200
Progress: 46/1200
Progress: 47/1200
Progress: 48/1200
Progress: 49/1200
Progress: 50/1200
Progress: 51/1200
Progress: 52/1200
Progress: 53/1

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".selected"}
  (Session info: chrome=80.0.3987.132)


In [93]:
df.head(158)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,-1,About Agari\n\n\nFounded by thought leaders of...,4.2,Agari\n4.2,"Raleigh, NC","Foster City, CA",51 to 200 employees,2009,Company - Private,Enterprise Software & Network Solutions,Information Technology,$25 to $50 million (USD),-1
1,Senior Data Scientist - Threat Intelligence Re...,-1,What you’ll be doing...\n\nThis role requires ...,3.7,Verizon\n3.7,"Cary, NC","New York, NY",10000+ employees,2000,Company - Public,Telecommunications Services,Telecommunications,$10+ billion (USD),"Amazon, Microsoft, Google"
2,Entry Level Data Scientist,-1,The Center for\nData Science at RTI Internatio...,4.3,RTI International\n4.3,"Durham, NC","Research Triangle Park, NC",1001 to 5000 employees,1958,Nonprofit Organization,Research & Development,Business Services,$500 million to $1 billion (USD),"Westat, Abt Associates, Chemonics International"
3,Data Scientist I,-1,LexisNexis is looking for a Data Scientist I.\...,3.5,Elsevier\n3.5,"Raleigh, NC","Amsterdam, Netherlands",5001 to 10000 employees,1880,Subsidiary or Business Segment,Publishing,Media,$2 to $5 billion (USD),"Clarivate Analytics, Springer Nature, Thomson ..."
4,Data Scientist Intern,-1,Infinia ML is looking for interns excited to d...,4.4,Infinia ML\n4.4,"Durham, NC","Durham, NC",1 to 50 employees,2017,Company - Private,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1
5,Data Scientist,-1,"As a Data Scientist, you will evaluate and imp...",3.5,Quadient\n3.5,"Raleigh, NC","Bagneux, France",5001 to 10000 employees,1924,Company - Public,Enterprise Software & Network Solutions,Information Technology,$1 to $2 billion (USD),"Thunderhead, OpenText, Acquia"
6,Applied Data Scientist,-1,THE POSITION\n\nLendingTree's Data Science tea...,3.8,LendingTree\n3.8,"Charlotte, NC","Charlotte, NC",1001 to 5000 employees,1996,Company - Public,Lending,Finance,$500 million to $1 billion (USD),"Zillow, Trulia"
7,Business Analytics Data Scientist,-1,"Thursday, April 25, 2019\n\nPosition: Business...",4.6,Carolina Panthers\n4.6,"Charlotte, NC","Charlotte, NC",51 to 200 employees,1989,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$10 to $25 million (USD),-1
8,Data Scientist,-1,growing rapidly and looking for exceptional ne...,4.8,Auth0\n4.8,"Charlotte, NC","Bellevue, WA",501 to 1000 employees,2014,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,-1
9,Data Scientist (All Levels),-1,Help build technology that saves lives!\nLeanT...,4.5,LeanTaaS\n4.5,"Charlotte, NC","Santa Clara, CA",51 to 200 employees,2010,Company - Private,Health Care Services & Hospitals,Health Care,$25 to $50 million (USD),-1


In [12]:
analyst_MA = get_jobs_GD("Media Analyst", 600, 'all', 'verbose')
analyst_MA.to_csv ('analyst_MA_all_3-10.csv',float_format="%1.2f")
analyst_MA

Initialized browser
Signed In
Opened Page
Progress: 0/600
Job Title: Social Media Marketing Analyst
Salary Estimate: -1
Job Description: Social Media Marketing Analyst


JOB DESCRIPTION

When scrolling through your social media feed, are you ever surprised at how timely and relevant the sponsored advertisements are? Have you ever wondered how those ads are created and why you were targeted for that specific campaign? Social Media platforms allow a unique window into the lives of consumers; you will be able to meet consumers where they are and help our clients grow their businesses by creating eye-catching, interesting, timely, an
Rating: 3.8
Company Name: ROI Revolution
3.8
Location: Raleigh, NC
Headquarters: Raleigh, NC
Size: 51 to 200 employees
Founded: 2002
Type of Ownership: Company - Private
Industry: Advertising & Marketing
Sector: Business Services
Revenue: Unknown / Non-Applicable
Competitors: -1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 1/600
Job Title: Pa

Headquarters: Hickory, NC
Size: 10000+ employees
Founded: 1965
Type of Ownership: Company - Public
Industry: Telecommunications Manufacturing
Sector: Telecommunications
Revenue: $2 to $5 billion (USD)
Competitors: -1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 9/600
Job Title: Sr. Analyst of Paid Social Media
Salary Estimate: -1
Job Description: PURPOSE OF ROLE:
The primary purpose of the Paid Media Sr. Analyst is to support the Paid Media Manager by delivering the daily execution and optimization of digital marketing campaigns. The Paid Media Sr. Analyst will support all aspects of digital marketing channel and campaign management, including day to day campaign stewardship, vendor partnership planning and collaboration, competitive tracking and insight, and campaign performance recaps and communication. The Paid Media Sr. Analyst is re
Rating: 2.8
Company Name: Ettain Group
2.8
Location: Mooresville, NC
Headquarters: Charlotte, NC
Size: 1001 to 5000 employees
Founde

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Social Media Marketing Analyst,-1,Social Media Marketing Analyst\n\n\nJOB DESCRI...,3.8,ROI Revolution\n3.8,"Raleigh, NC","Raleigh, NC",51 to 200 employees,2002,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,-1
1,Paid Media Analyst,-1,"As a Paid Media Analyst, you will be a critica...",3.2,Red Ventures\n3.2,"Charlotte, NC","Fort Mill, SC",1001 to 5000 employees,2000,Company - Private,Advertising & Marketing,Business Services,$1 to $2 billion (USD),"Clearlink, Credit Karma, LendingTree"
2,Paid Social Media Analyst,-1,"At Wildfire, our Strategy team is structured t...",3.9,Wildfire\n3.9,"Winston-Salem, NC","Winston Salem, NC",1 to 50 employees,-1,Company - Private,Advertising & Marketing,Business Services,$1 to $5 million (USD),-1
3,2020 Summer Internship - Data Analyst,-1,More than a career - a chance to make a differ...,3.6,Duke Energy\n3.6,"Charlotte, NC","Charlotte, NC",10000+ employees,1917,Company - Public,Energy,"Oil, Gas, Energy & Utilities",$10+ billion (USD),-1
4,Media Operations Specialist,-1,ConsumerTrack™ is unique in the digital market...,3.1,ConsumerTrack\n3.1,"Charlotte, NC","El Segundo, CA",51 to 200 employees,2004,Company - Private,Advertising & Marketing,Business Services,$25 to $50 million (USD),-1
5,Senior Media Planner / Media Supervisor,-1,Four-year degree is required.\nMinimum 4 years...,3.1,Mullen\n3.1,North Carolina,"Boston, MA",501 to 1000 employees,1970,Subsidiary or Business Segment,Advertising & Marketing,Business Services,$50 to $100 million (USD),"Droga5, Arnold"
6,"Digital Strategist, Paid Media",-1,Overview\n\nWe're looking for a Digital Strate...,3.6,"Ketchum, Inc.\n3.6","Raleigh, NC","New York, NY",1001 to 5000 employees,1923,Subsidiary or Business Segment,Advertising & Marketing,Business Services,Unknown / Non-Applicable,-1
7,Senior Media Buyer - Entertainment and Media I...,-1,We are seeking a Senior Media Buyer to join ou...,3.1,Mullen\n3.1,"Winston-Salem, NC","Boston, MA",501 to 1000 employees,1970,Subsidiary or Business Segment,Advertising & Marketing,Business Services,$50 to $100 million (USD),"Droga5, Arnold"
8,Unified Communication Telecom Analyst,-1,Everyone communicates. It’s the essence of the...,3.4,CommScope Inc.\n3.4,"Richardson, NC","Hickory, NC",10000+ employees,1965,Company - Public,Telecommunications Manufacturing,Telecommunications,$2 to $5 billion (USD),-1
9,Sr. Analyst of Paid Social Media,-1,PURPOSE OF ROLE:\nThe primary purpose of the P...,2.8,Ettain Group\n2.8,"Mooresville, NC","Charlotte, NC",1001 to 5000 employees,1996,Company - Private,Staffing & Outsourcing,Business Services,Unknown / Non-Applicable,-1
